### black_hole_properties
---
1. This script looks at how all black hole properties correlate with low and high mass galaxies of all morphologies.

---

In [281]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import split_population
import plot_population
import imp

imp.reload(split_population)
imp.reload(plot_population)
from astropy.cosmology import Planck15

In [2]:
from matplotlib import rc
rc('text', usetex=True)

In [302]:
outplot = '/Users/cd201/projects/bh_star_gas_misalignment/local/plots/'

In [3]:
# ---------------------------------------------------------------------------------------
# Loading in TNG100 sample with z=0 info to select samples.
tng100_main = pd.read_csv('/Users/cd201/morphology_misalignment/catalogues/tng100_mpl8_pa_info_v0.1_z0_info.csv', comment='#')
tng100_pa = split_population.tng100_pa_sample(tng100_main)

# Breaking down morphology (deviations from sfms).
QU, SF, GV = split_population.SFMS_breakdown(tng100_pa)

# ---------------------------------------------------------------------------------------
# Loading in mass history file.
mass_tab = pd.read_csv('/Users/cd201/projects/bh_star_gas_misalignment/popeye/catalogues/tng100_bh_history.csv')
mass_tab['branch_lookback_time'] = Planck15.lookback_time(mass_tab.branch_z.values).value
# ---------------------------------------------------------------------------------------

In [4]:
# defining new columns for mass table.
mass_tab['gas_fraction'] = mass_tab['gas_mass'] / mass_tab['stel_mass']
mass_tab['BH_CumEgyInjection_Total'] = mass_tab['BH_CumEgyInjection_QM'] + mass_tab['BH_CumEgyInjection_RM']

In [75]:
# Loading in ang mom history file.
ang_tab = pd.read_csv('/Users/cd201/morphology_misalignment/catalogues/tng_mpl8_ang_mom_history.csv')

branch_z = np.zeros(ang_tab.shape[0])
# adding columns to table?
for i, sn in enumerate(ang_tab.branch_snapnum.values):
    branch_z[i] = split_population.snap_to_z(sn)
ang_tab['branch_z'] = branch_z
ang_tab['branch_lookback_time'] = Planck15.lookback_time(ang_tab.branch_z.values).value

In [138]:
# adding new instantaneous energy injected column. this takes a while.

mass_tab['BH_InstantEgyInjection_Total'] = np.zeros(mass_tab.shape[0])
mass_tab['BH_InstantEgyInjection_QM'] = np.zeros(mass_tab.shape[0])
mass_tab['BH_InstantEgyInjection_RM'] = np.zeros(mass_tab.shape[0])

for sub in np.unique(mass_tab.root_subfind.values):
    print(sub)
    # defining single tree.
    single_tab = mass_tab[mass_tab.root_subfind.values == sub]
    # finding instantaneous energy output at each snapshot.
    cumsum_energy_QM = - single_tab.BH_CumEgyInjection_QM.diff()[1:]
    cumsum_energy_RM = - single_tab.BH_CumEgyInjection_RM.diff()[1:]
    cumsum_energy_total = cumsum_energy_QM + cumsum_energy_RM
    
    # adding additional value at start to compensate for size difference.
    cumsum_energy_QM = np.append(cumsum_energy_QM.values, 0)
    cumsum_energy_RM = np.append(cumsum_energy_RM.values, 0)
    cumsum_energy_total = np.append(cumsum_energy_total.values, 0)

    mass_tab['BH_InstantEgyInjection_Total'][mass_tab.root_subfind.values == sub] = cumsum_energy_total
    mass_tab['BH_InstantEgyInjection_QM'][mass_tab.root_subfind.values == sub] = cumsum_energy_QM
    mass_tab['BH_InstantEgyInjection_RM'][mass_tab.root_subfind.values == sub] = cumsum_energy_RM


3


/Users/cd201/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:21: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/cd201/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:22: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
/Users/cd201/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


4
5
12
13
15
16
17
18
19
20
21
23
26
27
29
31
33
34
36
38
39
44
45
46
47
48
49
50
52
56
57
59
60
65
67
70
71
73
74
75
76
77
78
79
81


/Users/cd201/anaconda3/lib/python3.6/site-packages/pandas/core/algorithms.py:1635: RuntimeWarning: invalid value encountered in subtract
  out_arr[res_indexer] = arr[res_indexer] - arr[lag_indexer]


83
85
86
90
91
93
94
95
97
99
100
103
107
109
110
111
112
114
117
120
121
124
127
128
129
130
131
133
137
145
146
147
149
150
154
155
157
159
160
162
165
166
167
168
178
184
188
190
197
200
206
216
229
234
246
248
262
273
278
758
17189
17191
17193
17194
17195
17196
17197
17198
17199
17200
17202
17203
17204
17205
17206
17208
17212
17214
17217
17218
17219
17221
17222
17226
17227
17229
17232
17233
17234
17235
17236
17238
17239
17240
17241
17244
17245
17247
17248
17249
17250
17251
17252
17253
17254
17255
17256
17257
17258
17263
17265
17266
17267
17268
17269
17271
17274
17275
17277
17278
17281
17284
17291
17293
17294
17296
17297
17302
17303
17304
17306
17308
17310
17311
17316
17323
17325
17330
17337
17340
17345
17347
17348
17362
17368
17399
17449
17453
17465
17495
17592
31345
31346
31347
31348
31349
31350
31351
31352
31353
31354
31355
31356
31357
31359
31360
31361
31362
31363
31364
31366
31367
31368
31371
31372
31373
31375
31377
31378
31380
31383
31384
31389
31391
31392
31393
31394
31396
31

191100
192899
192900
192901
192902
192903
192906
192907
192908
192909
192910
192912
192914
192918
192919
192920
192923
192924
192928
192933
194950
194958
194959
194961
194963
194965
194970
197110
197111
197112
197113
197115
197117
197120
197125
197135
197149
197155
199322
199323
199326
199327
199333
200804
200805
200806
200807
200808
200809
200810
200814
200818
200819
200820
200821
200829
200848
202635
202636
202637
202638
202639
202640
202642
202643
202644
202645
202646
202647
202648
202654
202659
204235
204237
204238
204240
204241
204243
204245
204252
204253
204255
204256
204258
204262
204265
204267
206228
206232
206236
207474
207475
207477
207478
207479
207480
207481
207487
207488
207489
207498
207520
208783
208784
208787
208792
208793
208797
208800
210763
210766
210767
210768
210771
210777
210779
210780
210786
210788
212837
212838
212840
212841
212844
212846
212847
212848
212853
212855
214453
214454
214456
214457
214458
214459
214460
214463
214467
214483
214530
216017
216018
216024

362373
362551
362552
362554
362559
362780
362782
363104
363105
363107
363315
363317
363318
363526
363531
363736
363737
363738
363744
364263
364479
364482
364483
364807
364815
365010
365493
365743
365745
366035
366036
366037
366038
366040
366338
366341
366345
366351
366565
366567
366792
366793
367139
367456
367457
367688
367689
367934
368170
368174
368177
368438
368697
368698
368700
368884
368898
369152
369156
369366
369367
369368
369545
369546
369789
370003
370004
370006
370008
370190
370191
370192
370195
370529
370781
370782
371013
371016
371259
371471
371477
371479
371704
371705
371707
371710
371859
372175
372176
372343
372345
372568
372778
372786
373050
373053
373291
373292
373295
373510
373511
373699
373700
373702
373951
373952
373954
374228
374229
374379
374385
374388
374593
374789
375236
375237
375401
375403
375404
375410
375639
375641
375909
375911
376132
376135
376356
376357
376544
376545
376554
376738
376740
376741
376963
377212
377214
377539
377543
377804
378116
378298
378300

476527
476528
476557
476624
476820
476892
477032
477119
477227
477419
477422
477518
477624
477875
477919
478092
478093
478160
478247
478358
478410
478452
478496
478497
478535
478604
478661
478927
479002
479314
479373
479411
479560
479561
479628
479799
479839
479884
479885
479917
480000
480075
480194
480285
480345
480391
480498
480550
480645
480712
480750
480879
480931
481036
481126
481254
481347
481405
481597
481894
481947
482006
482054
482118
482169
482196
482257
482481
482533
482714
482814
482815
482869
482987
483068
483113
483179
483216
483269
483421
483484
483693
483749
483812
483868
483900
483901
483977
483978
484045
484113
484155
484257
484307
484427
484608
484609
484715
484905
484944
485095
485233
485283
485284
485326
485365
485485
485597
485703
485747
485887
485951
486009
486052
486124
486186
486281
486341
486384
486463
486782
486875
486923
487042
487072
487099
487100
487152
487244
487276
487363
487364
487438
487496
487698
487877
487913
487965
487993
488228
488356
488446
488537

583480
583580
583740
583824
583919
584104
584313
584357
584378
584710
584885
585026
585123
585290
585302
585335
585369
585445
585553
585575
585609
585689
585860
585970
586131
586279
586487
586541
586561
586587
586749
587096
587122
587293
587357
587771
587792
587917
588209
588239
588251
588295
588397
588403
588523
588554
588670
588676
588772
589031
589046
589115
589351
589507
589541
589573
589611
589669
589677
589743
589988
590104
590158
590401
590511
590583
590890
590992
591106
591265
591278
591288
591344
591425
591604
591612
591668
591774
591922
591953
592022
592046
592283
592302
592324
592368
592594
592690
592691
592734
592847
592962
593039
593051
593121
593264
593271
593302
593397
593430
593493
593499
593509
593523
593572
593573
593595
593747
593832
594444
594459
594499
594532
594757
594800
594892
594991
595031
595058
595171
595201
595238
595336
595380
595460
595498
595586
595799
595832
595977
595993
596057
596113
596132
596326
596385
596430
596678
596929
597066
597117
597121
597135

In [149]:
# dropping all mass table columns with nan.

mass_tab = mass_tab.dropna(subset=['BH_InstantEgyInjection_Total'])
mass_tab = mass_tab[(mass_tab.BH_InstantEgyInjection_Total < np.inf) & (mass_tab.BH_InstantEgyInjection_Total > -np.inf)]

In [303]:
plt.cla()
plt.close()
fig, ax = plt.subplots(5, 3, figsize=(13, 17), sharey='row', sharex='all')
plot_population.plot_row_evolution(QU, GV, SF, mass_tab, 'log10_Lbh_bol', 0, ax[0, :], 33, 66)
plot_population.plot_row_evolution(QU, GV, SF, mass_tab, 'gas_fraction', 0, ax[1,:], 33, 66)
plot_population.plot_row_evolution(QU, GV, SF, ang_tab, 'mag_sJ_gas', 0, ax[2,:], 33, 66)
plot_population.plot_row_evolution(QU, GV, SF, mass_tab, 'GasMetallicity', 0, ax[3,:], 33, 66)
# central star formation?
plot_population.plot_row_evolution(QU, GV, SF, mass_tab, 'SFR', 0, ax[4,:], 33, 66)
#plot_population.plot_row_residual_HM(QU, GV, SF, mass_tab, 'BH_mass', 0, ax[5,:], 33, 66)

ax[0,0].set_ylabel(r'$log10(L_{bol})$ [ergs/s]', fontsize=14)
ax[1,0].set_ylabel(r'Gas fraction, [$M_{gas}$/$M_{stel}$]', fontsize=14)
ax[2,0].set_ylabel(r'$j_{gas}$ [km/s $\cdot$ kpc]', fontsize=14)
ax[3,0].set_ylabel(r'Gas phase Metallicity, Z', fontsize=14)
ax[4,0].set_ylabel(r'sSFR, [$M_{\odot}/Yr$]', fontsize=14)

ax[0,0].set_title('Quenched at $z=0$', fontsize=14)
ax[0,1].set_title('Green valley at $z=0$', fontsize=14)
ax[0,2].set_title('Star forming at $z=0$', fontsize=14)

ax[-1,0].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[-1,1].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[-1,2].set_xlabel('lookback time, [Gyr]', fontsize=14)

ax[0,0].legend(frameon=False, fontsize=12)
ax[0,1].legend(frameon=False, fontsize=12)

ax[0,0].set_ylim([40.05, 44.5])
plot_population.xtick_format(1, 0.5, ax[0,0], format='%1.0f')
plot_population.ytick_format(1, 0.5, ax[0,0], format='%1.0f')

ax[1,0].set_yscale('log')
ax[2,0].set_yscale('log')

ax[3,0].set_ylim([0.003, 0.035])
plot_population.ytick_format(0.01, 0.005, ax[3,0], format='%1.2f')

ax[4,0].set_yscale('log')
#ax[5,0].set_yscale('log')

fig.subplots_adjust(wspace=0, hspace=0)
plt.savefig(outplot+'overall_pop_evolution.pdf', format='pdf', bbox_inches='tight')

/Users/cd201/projects/bh_star_gas_misalignment/local/lib/plot_population.py:70: RuntimeWarning: invalid value encountered in greater
  plot_property_evolution(QU_align_tab_HM.branch_lookback_time.values[QU_align_tab_HM[property].values > condition], QU_align_tab_HM[property].values[QU_align_tab_HM[property].values > condition], ax[0], r'Top 33\% $M_{stel}$, $\Delta$PA $< 30^{\circ}$', color='salmon', linestyle='solid')
/Users/cd201/projects/bh_star_gas_misalignment/local/lib/plot_population.py:71: RuntimeWarning: invalid value encountered in greater
  plot_property_evolution(QU_mis_tab_HM.branch_lookback_time.values[QU_mis_tab_HM[property].values > condition], QU_mis_tab_HM[property].values[QU_mis_tab_HM[property].values > condition], ax[0], r'Top 33\% $M_{stel}$, $\Delta$PA $\geq 30^{\circ}$',color='salmon', linestyle='dashed')
/Users/cd201/projects/bh_star_gas_misalignment/local/lib/plot_population.py:75: RuntimeWarning: invalid value encountered in greater
  plot_property_evolution(

In [332]:
fig, ax = plt.subplots(2,3, figsize=(15, 7), sharex='all')
plot_population.plot_row_residual_LM(QU, GV, SF, mass_tab, 'BH_InstantEgyInjection_Total', 0, ax[0,:], 33, 66)
plot_population.plot_row_residual_LM(QU, GV, SF, mass_tab, 'BH_mass', 0, ax[1,:], 33, 66)

ax[0,0].legend(frameon=False, fontsize=12, loc='upper left')
ax[0,0].set_ylim([-10**15, 3*10**15])
ax[0,1].set_ylim([-10**15 / 3, 10**15])
ax[0,2].set_ylim([-10**14, 3 * 10**14])

ax[1,0].set_ylim([-10**7, 3*10**7])
ax[1,1].set_ylim([-10**7 /1.5, 1.5*10**7])
ax[1,2].set_ylim([-10**6, 3*10**6])

ax[0,0].set_ylabel('$\Delta$Energy injected, \n [$M_{\odot} (ckpc/h)^{2} / (0.978 Gyr/h)^{2}$]', fontsize=12)
ax[0,1].set_ylabel('$\Delta$Energy injected, \n [$M_{\odot} (ckpc/h)^{2} / (0.978 Gyr/h)^{2}$]', fontsize=12)
ax[0,2].set_ylabel('$\Delta$Energy injected, \n [$M_{\odot} (ckpc/h)^{2} / (0.978 Gyr/h)^{2}$]', fontsize=12)

ax[1,0].set_ylabel('$\Delta$$M_{BH}$, [$M_{\odot}$]', fontsize=12)
ax[1,1].set_ylabel('$\Delta$$M_{BH}$, [$M_{\odot}$]', fontsize=12)
ax[1,2].set_ylabel('$\Delta$$M_{BH}$, [$M_{\odot}$]', fontsize=12)

ax[0,0].set_title('Quenched at $z=0$', fontsize=14)
ax[0,1].set_title('Green valley at $z=0$', fontsize=14)
ax[0,2].set_title('Star forming at $z=0$', fontsize=14)


ax[0,0].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[0,1].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[0,2].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[0,0].xaxis.set_tick_params(labelbottom=True)
ax[0,1].xaxis.set_tick_params(labelbottom=True)
ax[0,2].xaxis.set_tick_params(labelbottom=True)

ax[-1,0].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[-1,1].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[-1,2].set_xlabel('lookback time, [Gyr]', fontsize=14)

plot_population.xtick_format(1, 0.5, ax[0,0], format='%1.0f')
ax[1,2].ticklabel_format(style='sci', axis='y', scilimits=(0,0))

fig.subplots_adjust(wspace=0.35, hspace=0.25)
plt.savefig(outplot+'LM_BH_residual_evo.pdf', format='pdf', bbox_inches='tight')

In [334]:
fig, ax = plt.subplots(2,3, figsize=(15, 7), sharex='all')
plot_population.plot_row_residual_HM(QU, GV, SF, mass_tab, 'BH_InstantEgyInjection_Total', 0, ax[0,:], 33, 66)
plot_population.plot_row_residual_HM(QU, GV, SF, mass_tab, 'BH_mass', 0, ax[1,:], 33, 66)

ax[0,0].legend(frameon=False, fontsize=12, loc='upper left')
ax[0,0].set_ylim([-10**16, 3*10**16])
ax[0,1].set_ylim([-10**16 / 3, 10**16])
ax[0,2].set_ylim([-10**15, 3 * 10**15])

ax[1,0].set_ylim([-10**8, 3*10**8])
ax[1,1].set_ylim([-10**8 /1.5, 1.5*10**8])
ax[1,2].set_ylim([-10**8 /1.5, 1.5*10**8])

ax[0,0].set_ylabel('$\Delta$Energy injected, \n [$M_{\odot} (ckpc/h)^{2} / (0.978 Gyr/h)^{2}$]', fontsize=12)
ax[0,1].set_ylabel('$\Delta$Energy injected, \n [$M_{\odot} (ckpc/h)^{2} / (0.978 Gyr/h)^{2}$]', fontsize=12)
ax[0,2].set_ylabel('$\Delta$Energy injected, \n [$M_{\odot} (ckpc/h)^{2} / (0.978 Gyr/h)^{2}$]', fontsize=12)

ax[1,0].set_ylabel('$\Delta$$M_{BH}$, [$M_{\odot}$]', fontsize=12)
ax[1,1].set_ylabel('$\Delta$$M_{BH}$, [$M_{\odot}$]', fontsize=12)
ax[1,2].set_ylabel('$\Delta$$M_{BH}$, [$M_{\odot}$]', fontsize=12)

ax[0,0].set_title('Quenched at $z=0$', fontsize=14)
ax[0,1].set_title('Green valley at $z=0$', fontsize=14)
ax[0,2].set_title('Star forming at $z=0$', fontsize=14)


ax[0,0].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[0,1].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[0,2].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[0,0].xaxis.set_tick_params(labelbottom=True)
ax[0,1].xaxis.set_tick_params(labelbottom=True)
ax[0,2].xaxis.set_tick_params(labelbottom=True)

ax[-1,0].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[-1,1].set_xlabel('lookback time, [Gyr]', fontsize=14)
ax[-1,2].set_xlabel('lookback time, [Gyr]', fontsize=14)

plot_population.xtick_format(1, 0.5, ax[0,0], format='%1.0f')
ax[1,2].ticklabel_format(style='sci', axis='y', scilimits=(0,0))

fig.subplots_adjust(wspace=0.35, hspace=0.25)
plt.savefig(outplot+'HM_BH_residual_evo.pdf', format='pdf', bbox_inches='tight')